In [1]:
import pandas as pd
dataset = pd.read_csv('covid-data.csv')

In [2]:
df = dataset.copy()
datasetArray = df.values



In [3]:
arr = []

start = 3192 #2495
end = 3353 #2637

for i in range(start, end):
    arr.append(datasetArray[i])

infected = []
deaths = []
for i in range(len(arr)):
    infected.append(arr[i][5])
    deaths.append(arr[i][7])
    
days = []
from numpy import *
for i in range(len(infected)):
    days.append(i+1)
    if isnan(infected[i]):
        infected[i] = 0
    if isnan(deaths[i]):
        deaths[i] = 0

%matplotlib inline
import matplotlib.pyplot as mpl
# mpl.bar(days, infected)
# print(days)
# mpl.plot(days, deaths)

In [4]:
beta = []
gamma = []

def getGamma(r0, r1, x0):
    if x0 != 0:
        return (r1 - r0) / x0
    else:
        return 0

def getBeta(x0, x1, r0, r1):
    if x0 != 0:
        return ((x1 - x0) + (r1 - r0)) / x0
    else:
        return 0

for i in range(len(arr) - 1):
    gamma.append(getGamma(deaths[i], deaths[i+1], infected[i]))
    beta.append(getBeta(infected[i], infected[i+1], deaths[i], deaths[i+1]))
    
# print(days)
    
# mpl.plot(days[:-1], beta)

# mpl.plot(days[:-1], beta)

In [5]:
from sklearn.linear_model import Ridge
from pandas import DataFrame

import numpy as np

# beta = np.asarray(beta, dtype = np.float32)
# days = np.asarray(days)
# print(days)
print(beta)

lastBeta = beta[len(beta)-1]
lastGamma = gamma[len(gamma)-1]
print(lastGamma)

def convert(arr):
    finalArr = []
    for i in range(len(arr)):
        temp = []
        temp.append(arr[i])
        finalArr.append(temp)
        
    return finalArr
beta = convert(beta)
gamma = convert(gamma)
cDays = convert(days[:-1])


[0, 0, 0, -1.0, 0, 0, 0, 0, 0, 0, 0, 0.5, -0.3333333333333333, -1.0, 0, 0.2857142857142857, -0.7142857142857143, 0.6666666666666666, 0.16666666666666666, -1.0, 0, -1.0, 0, 0, -1.0, 0, 1.0, -0.6666666666666666, 1.5, 1.2, 0.8888888888888888, 1.0555555555555556, 0.22857142857142856, 0.2682926829268293, 1.037037037037037, -0.11607142857142858, -0.4148936170212766, 1.4137931034482758, 0.31654676258992803, 0.15934065934065933, 0.03349282296650718, 0.5844748858447488, -0.20527859237536658, 0.12781954887218044, 0.013071895424836602, 0.5865384615384616, -0.11991869918699187, -0.09907834101382489, 0.05384615384615385, 0.20772946859903382, -0.3757455268389662, 0.33980582524271846, 0.1937799043062201, 0.096579476861167, 0.1766848816029144, -0.12480499219968799, 0.0070921985815602835, -0.028021015761821366, 0.19927536231884058, 0.039097744360902256, 0.13662790697674418, 0.007633587786259542, -0.09367088607594937, -0.00424929178470255, -0.10155148095909731, 0.40408805031446543, 0.16234498308906425, 

In [7]:
betaModel = Ridge().fit(cDays, beta)

gammaModel = Ridge().fit(cDays, gamma)

In [9]:
days_test = []
for i in range(10):
    day = 160 + i + 1
    days_test.append(np.array([[day]]))

pBetas = []
pGammas = []
for i in range(5):
    pbeta = betaModel.predict(days_test[i])
    pgamma = gammaModel.predict(days_test[i])
    pBetas.append(pbeta[0][0])
    pGammas.append(pgamma[0][0])
    
pBetas.insert(0, lastBeta)
pGammas.insert(0, lastGamma)
print(pGammas)
print(pBetas)
print('MSE: ', betaModel.best_score_)

[-0.011014686248331108, -0.003273927252195514, -0.0033592938699309616, -0.003444660487666411, -0.003530027105401859, -0.0036153937231373084]
[-1.010347129506008, 0.0028559272348545883, 0.0023213819295017923, 0.0017868366241489964, 0.0012522913187962004, 0.0007177460134433905]


AttributeError: 'Ridge' object has no attribute 'best_score_'

In [44]:
import copy

def getInfPrediction(day, pBetas, pGammas):
    lastInf = infected[(len(infected) - 2)]
    interval = abs((days[len(days)-1] - day))
    print(interval, lastInf)
    newInfs = []
    for i in range(interval):
        print(lastInf)
        factor = (1 + pBetas[i] - pGammas[i])
        print('factor', factor)
        nextInf = factor * lastInf
        newInfs.append(nextInf)
        lastInf=copy.deepcopy(nextInf)
    return newInfs

pInfs = getInfPrediction(165, pBetas, pGammas)
print(pInfs)

4 2996.0
2996.0
factor 0.0006675567423230847
1.9999999999999616
factor 1.00612985448705
2.0122597089740615
factor 1.0056806757994328
2.023690704005004
factor 1.0052314971118153
[1.9999999999999616, 2.0122597089740615, 2.023690704005004, 2.0342776360782135]
